# KOSPI 반도체 섹터 기업 주가, 거래량 크롤링 from Naver Finance

증권사 API를 쓰지 않고 크롤링으로 주가 데이터를 가져올 때 수정종가(Adjusted Close)를 가져오는 것이 큰 난관이다. 

네이버의 일별시셰 페이지는 단순 시가를 보여주고 수정종가를 보여주지 않기 때문에 네이버의 차트 페이지의 xml페이지를 불러 데이터를 가져와야 한다. 

검색해보니 종전에는 다음(카카오) 파이넌스에서 수정종가를 제공하여 크롤링을 많이 했다고 하는데, 현재는 http request를 통한 시세 페이지 접근을 막은 것으로 보인다. 

http://henryquant.blogspot.com/2018/11/r.html 를 참고하였다. 

In [83]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs

우선 네이버 파이넌스의 '반도체와 반도체장비' 업종 내 종목들 중 KOSPI 종목들만 ticker을 가져온다. 

페이지의 html 내에서 ticker는 쉽게 가져올 수 있지만 KOSPI/KOSDAQ 여부는 조금 번거롭지만 종목명 뒤에 *이 달려있는지 달려있지 않은지를 통해 필터해야 한다. 

In [55]:
semiconductor_html = urlopen("https://finance.naver.com/sise/sise_group_detail.nhn?type=upjong&no=202")
semiconductor_bs = bs(semiconductor_html.read(), 'html.parser')

In [117]:
ticker_dic = {}

for tr in semiconductor_bs.find('table', {'summary':'업종별 시세 리스트'}).find_all('tr')[1:]:
    if not tr.td.span:
        continue
    
    if tr.td.span.get_text() != '*': # 코스닥 종목은 *. 코스피는 *가 없다. 
        ticker_dic[tr.td.a.attrs['href'][-6:]] = tr.td.a.get_text() # Naver의 반도체와 반도체 장비 업종 내 KOSPI 종목들 ticker 딕셔너리 eg: '003160':'디아이'

In [118]:
ticker_dic

{'003160': '디아이',
 '077500': '유니퀘스트',
 '281820': '케이씨텍',
 '000990': 'DB하이텍',
 '005930': '삼성전자',
 '042700': '한미반도체',
 '000660': 'SK하이닉스',
 '029460': '케이씨',
 '000995': 'DB하이텍1우',
 '005935': '삼성전자우',
 '195870': '해성디에스',
 '092220': 'KEC',
 '006200': '한국전자홀딩스',
 '025560': '미래산업'}

이제 각 종목의 20161207 부터의 일별 데이터를 가져온다. (날짜, 수정종가) 

거래량, 고가, 저가 등도 가져올 수 있으나 dataframe을 어떤 구조로 짜서 어떻게 사용할 것인지 생각해봐야 한다. 따라서 일단 수정 종가로만 하기로 한다. 

In [136]:
daycount = 570 # from 20161227 ~ 20190429
df = pd.DataFrame()

for ticker, name in ticker_dic.items():
    stock_xml_url = "https://fchart.stock.naver.com/sise.nhn?symbol={ticker}&timeframe=day&count={daycount}&requestType=0".format(ticker=ticker, daycount=str(daycount)) 
    
    html = urlopen(stock_xml_url)
    html_bs = bs(html.read(), 'html.parser')
    
    temp_df = pd.DataFrame(columns=['date', name])

    for i in html_bs.find_all('item'):
        data = i.attrs['data'].split("|") # 날짜, 시가, 고가, 저가, 종가, 거래량
        data = {'date':data[0], name:data[4]} # adj.close 
        temp_df = temp_df.append(data, ignore_index=True)
    
    if df.empty:
        df = temp_df
    else:
        df = df.merge(temp_df, on='date', how='left') # how='inner'일 경우 20171205~. 케이씨텍 때문. 

In [138]:
df.head()

,date,디아이,유니퀘스트,케이씨텍,DB하이텍,삼성전자,한미반도체,SK하이닉스,케이씨,DB하이텍1우,삼성전자우,해성디에스,KEC,한국전자홀딩스,미래산업
0,20161227,4655,7350,NaN,16000,35980,6000,45650,12958,29750,28560,12400,996,915,291
1,20161228,4700,7020,NaN,15650,35760,6180,45350,13000,29600,28480,12750,1005,922,286
2,20161229,4705,7330,NaN,15900,36040,6180,44700,13417,29700,28660,12950,1000,931,283
3,20170102,4730,7160,NaN,16300,36100,6140,45800,13458,30550,28800,13000,1020,922,289
4,20170103,4900,7060,NaN,17250,36480,6160,47250,13208,30950,28780,13150,1035,932,290
